In [4]:
from stas_dedup import deduplicate_dataset

ImportError: cannot import name 'CommitInfo' from 'huggingface_hub' (/Users/waynelau/miniconda3/envs/text-dedup/lib/python3.11/site-packages/huggingface_hub/__init__.py)